In [3]:
import os 
dirname = os.path.abspath('')



In [1]:
os.path.join(dirname, "../pretrained_models/ukdale/KNNweights_max20comp ukdale_1hr_16_19_08-27_05_2022.pkl")

NameError: name 'dirname' is not defined

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import ExtraTreeClassifier
from skmultilearn.adapt import MLkNN
from skmultilearn.ensemble import RakelD

from datasources.paths_manager import SAVED_MODEL, MY_SAVED_MODEL, PATH_SIGNAL2VEC, MY_PATH_SIGNAL2VEC
from nilmlab.factories import TransformerFactory
from nilmlab.lab import TransformerType


SIGNAL2VEC = 'SIGNAL2VEC'
MYSIGNAL2VEC = 'MYSIGNAL2VEC'
TRANSFORMER_MODELS = 'TRANSFORMER_MODELS'
CLF_MODELS = 'CLF_MODELS'

In [ ]:
# Build embeddings
gmm_experiment = {
    MYSIGNAL2VEC : {
        CLF_MODELS : [
            MLPClassifier(hidden_layer_sizes=(2000, 100, 100), learning_rate='adaptive', solver='adam')
        ],
        TRANSFORMER_MODELS: [
            TransformerFactory.build_mysignal2vec_train()
        ]
    }
}

In [ ]:
# Run experiment
my_experiment = {
    MYSIGNAL2VEC : {
        CLF_MODELS : [
            MLPClassifier(hidden_layer_sizes=(2000, 100, 100), learning_rate='adaptive', solver='adam')
        ],
        TRANSFORMER_MODELS: [
            TransformerFactory.build_mysignal2vec_infer(MY_SAVED_MODEL, MY_PATH_SIGNAL2VEC, num_of_vectors=1)
        ]
    }
}